In [ ]:
#!pip uninstall -y torch torchvision torchaudio accelerate transformers sentence_transformers
#!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 accelerate==0.25.0 transformers==4.36.0 sentence_transformers==2.7.0
#!pip install eland elasticsearch

In [ ]:
#!pip install sentencepiece

In [ ]:
import sentencepiece
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

In [ ]:
# 1. ES 클라이언트 정의
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

load_dotenv()

url = os.getenv("ELASTIC_CLOUD_URL")
api_id = os.getenv("ELASTIC_API_ID")
api_key = os.getenv("ELASTIC_API_KEY")

client = Elasticsearch(
    url,
    api_key=(api_id, api_key),
    request_timeout=9000,
    max_retries=3,
    verify_certs=False
)

print(client.info())

In [ ]:
# 2. 임베딩 모델 다운로드
hf_model_id='intfloat/multilingual-e5-base'
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding")

In [ ]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

In [ ]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

In [ ]:
# 임베딩 모델 업로드 및 deploy
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)
print("model upload complete")
MlClient.start_trained_model_deployment(client, model_id=es_model_id)

In [ ]:
# 5. ES cluster에 배포된 Embedding 모델 확인
stats = MlClient.get_trained_models_stats(client, model_id=es_model_id)
stats.body['trained_model_stats'][0]['model_id']

In [5]:
es_model_id = 'intfloat__multilingual-e5-base'
# 6. 샘플 embedding
docs = [
    {
        "text_field": "안녕하세요. 사과 입니다."
    }
 ]

result = MlClient.infer_trained_model(client, model_id=es_model_id, docs=docs)

doc_0_vector = result['inference_results'][0]['predicted_value']
print(len(doc_0_vector))

doc_0_vector

d:\workspace\repos_python\rag-teampjt-2025\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'a30a7c8ef0bf435a9d350006622225d8.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


768


[0.03944661095738411,
 0.03974313288927078,
 0.02145634964108467,
 0.041385382413864136,
 0.0018388991011306643,
 -0.05730891227722168,
 -0.01979999803006649,
 -0.025851069018244743,
 -0.004950840026140213,
 0.0004533877654466778,
 -0.01400098018348217,
 -0.006592192687094212,
 0.2000322937965393,
 0.041858091950416565,
 -0.04186349734663963,
 -0.04744343087077141,
 0.01096693892031908,
 -0.01569594442844391,
 0.061558835208415985,
 -0.008899171836674213,
 0.022572575137019157,
 -0.012541813775897026,
 0.05584011599421501,
 -0.016366206109523773,
 0.05743333697319031,
 -0.016329804435372353,
 -0.005295100621879101,
 -0.0039400095120072365,
 -0.04492020234465599,
 0.046405937522649765,
 0.03135465458035469,
 -0.018744254484772682,
 0.02732716128230095,
 0.03319738805294037,
 -0.007591288536787033,
 0.01839747466146946,
 0.01078080479055643,
 -0.06724799424409866,
 0.020589614287018776,
 0.005073177628219128,
 0.02516796812415123,
 0.009052031673491001,
 0.022833624854683876,
 -0.0321001